In [ ]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance
from VRP.compare_solutions import num_different_edges


from proj_utils.path import print_path

from solver_step import SolverStep

import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.VRP_model import VRPSolutionModel
from multi_agent.solution_pool import solution_pool_simple
%matplotlib widget

In [ ]:
truck_volume_capacity = 15
truck_weight_capacity = 10
truck_capacity = (truck_volume_capacity, truck_weight_capacity)
dists, demand, points = generator(40)

In [ ]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [ ]:
solver_step_SA = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    )

In [ ]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    taboo_list_size
    )

In [ ]:
init_state = nearest_neighbour_solution(dists, demand, truck_capacity)
init_step = SolverStep(init_state,eval_function(init_state),5)
step_function_list = [solver_step_SA, solver_step_TS]

In [ ]:
solution_pool = solution_pool_simple(num_different_edges)
sol_model = VRPSolutionModel(init_step, step_function_list, None)

In [ ]:
for _ in range(100):
    sol_model.step()

In [ ]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states)

In [1]:
from VRP.compare_solutions import num_different_edges